<a href="https://colab.research.google.com/github/juanprida/nlp_with_transformers/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.4 MB/s eta 0:00:00


In [3]:
# General imports
import pandas as pd

# 🤗 Datasets

In [4]:
from datasets import list_datasets, load_dataset

In [5]:
# Check some datasets.
ds = list_datasets()
print(f"5 firsts Datasets available{ds[:5]}")

# Load emotion dataset.
emotions = load_dataset("emotion")
print(f"keys in the Dataset are {emotions.keys()}")

5 firsts Datasets available['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus']


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

keys in the Dataset are dict_keys(['train', 'validation', 'test'])


In [6]:
# Keep only train Dataset for now.
ds_train = emotions["train"]
print(f"First element in the train set {ds_train[0]}")

First element in the train set {'text': 'i didnt feel humiliated', 'label': 0}


In [7]:
# Pandas conversion
emotions.set_format(type="pandas")
df_train = emotions["train"][:]
df_train.head()

# Add label nanme
def get_label_name(label):
    """Get label name associated to encoded label."""
    return emotions["train"].features["label"].names[label]

df_train["label_name"] = df_train["label"].apply(get_label_name)

# 🤗Tokenizers

In [8]:
from transformers import AutoTokenizer

In [22]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "This is a silly test for a silly tokenizer."
encoded_text = tokenizer(text)
print(encoded_text)
print(tokenizer.convert_ids_to_tokens(encoded_text.input_ids))

{'input_ids': [101, 2023, 2003, 1037, 10021, 3231, 2005, 1037, 10021, 19204, 17629, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'a', 'silly', 'test', 'for', 'a', 'silly', 'token', '##izer', '.', '[SEP]']


In [21]:
# Tokenizing the whole dataset


{'input_ids': [101, 2023, 2003, 1037, 10021, 3231, 2005, 1037, 10021, 19204, 17629, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'a', 'silly', 'test', 'for', 'a', 'silly', 'token', '##izer', '.', '[SEP]']


['[CLS]', 'this', 'is', 'a', 'silly', 'test', 'for', 'a', 'silly', 'token', '##izer', '.', '[SEP]']
